In [1]:
from numba import jit

PATH_JSON_DATASET = "..\\..\\data\\legal_moves\\legal_moves_dataset_346894.json" # contains many pairs of "fen" and "legal_moves"

import json

def load_json_dataset(path):
    with open(path, "r") as file:
        return json.load(file)
    
dataset_json = load_json_dataset(PATH_JSON_DATASET)
                                
# Shuffle the dataset
import random
random.shuffle(dataset_json)

print("type(dataset_json): ", type(dataset_json))
print("type(dataset_json[0]): ", type(dataset_json[0]))
print("Number of pairs in the dataset: ", len(dataset_json))

print("Example of a few pairs: ")
for i in range(5):
    print(dataset_json[i])


type(dataset_json):  <class 'list'>
type(dataset_json[0]):  <class 'dict'>
Number of pairs in the dataset:  346894
Example of a few pairs: 
{'fen': '3B3K/8/2k1P3/p7/P5p1/1p6/2B1r3/8 w - - 3 80', 'legal_moves': ['h8g8', 'h8h7', 'h8g7', 'd8e7', 'd8c7', 'd8f6', 'd8b6', 'd8g5', 'd8a5', 'd8h4', 'c2h7', 'c2g6', 'c2f5', 'c2e4', 'c2d3', 'c2b3', 'c2d1', 'c2b1', 'e6e7']}
{'fen': '8/8/7B/8/5r2/8/1K2k2p/1B6 w - - 2 121', 'legal_moves': ['h6f8', 'h6g7', 'h6g5', 'h6f4', 'b2c3', 'b2b3', 'b2a3', 'b2c2', 'b2a2', 'b2c1', 'b2a1', 'b1h7', 'b1g6', 'b1f5', 'b1e4', 'b1d3', 'b1c2', 'b1a2']}
{'fen': '6b1/1R4K1/5B2/7k/8/8/B7/8 w - - 56 138', 'legal_moves': ['g7h8', 'g7g8', 'g7f8', 'b7b8', 'b7f7', 'b7e7', 'b7d7', 'b7c7', 'b7a7', 'b7b6', 'b7b5', 'b7b4', 'b7b3', 'b7b2', 'b7b1', 'f6d8', 'f6e7', 'f6g5', 'f6e5', 'f6h4', 'f6d4', 'f6c3', 'f6b2', 'f6a1', 'a2g8', 'a2f7', 'a2e6', 'a2d5', 'a2c4', 'a2b3', 'a2b1']}
{'fen': '5rk1/1qrb2p1/1p4Q1/p2Pp1Pp/P1p2R1P/1RP1p3/1P1B2P1/K4BNN w - - 1 30', 'legal_moves': ['g6e8', 'g6h7', '

In [2]:
# Convert the dataset in a format suitable for training

import torch
import chess

import sys
sys.path.append("..\\..\\")

from torrechess.utils import get_uci_move_by_index
from torrechess.utils import get_uci_move_index
from torrechess.utils import chessboard_to_tensorboard_29x8x8

def generate_training_data(dataset_json: dict[str, str]):
    board_tensors = []
    legal_moves_tensors = []
    for pair in dataset_json:
        # Convert the FEN string to a board
        board_from_white_perspective = chess.Board(pair["fen"])

        # Only accept WHITE to play datapoints
        if board_from_white_perspective.turn == chess.BLACK:
            continue

        # Convert the FEN string to a board then to the tensor
        board_tensors.append(chessboard_to_tensorboard_29x8x8(board_from_white_perspective))

        # Convert the legal moves to the tensor size 1858 given the uci mapping
        legal_moves = pair["legal_moves"]
        legal_moves_tensor = torch.zeros(1858)
        for i in range(1858):
            uci = get_uci_move_by_index(i)
            if uci in legal_moves:
                legal_moves_tensor[i] = 1
        legal_moves_tensors.append(legal_moves_tensor)
    return board_tensors, legal_moves_tensors

# Test with reduced dataset
#board_tensors, legal_moves_tensors = generate_training_data(dataset_json[:1])

#from torrechess.utils import print_pretty_tensor_Nx8x8
#print_pretty_tensor_Nx8x8(board_tensors[0])

# Print the legal moves according to the legal_moves_tensor
#def print_legal_moves(legal_moves_tensor):
#    for i in range(1858):
#        if legal_moves_tensor[i] == 1:
#            print(get_uci_move_by_index(i))
#
#print_legal_moves(legal_moves_tensors[0])


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\development\torrechess\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\development\torrechess\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\development\torrechess\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\development\to

In [3]:
# Generate training dataset for the whole dataset
board_tensors, legal_moves_tensors = generate_training_data(dataset_json)

print("len(board_tensors): ", len(board_tensors))
print("len(legal_moves_tensors): ", len(legal_moves_tensors))


len(board_tensors):  173701
len(legal_moves_tensors):  173701


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torrechess.utils import get_uci_move_by_index

# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

class ChessDataset(Dataset):
    def __init__(self, board_tensors, legal_moves_tensors):
        self.board_tensors = board_tensors
        self.legal_moves_tensors = legal_moves_tensors

    def __len__(self):
        return len(self.board_tensors)

    def __getitem__(self, idx):
        board_tensor = self.board_tensors[idx]
        legal_moves_tensor = self.legal_moves_tensors[idx]
        return board_tensor, legal_moves_tensor

class TorreEngineLegalMovesNN(nn.Module):
    def __init__(self):
        super(TorreEngineLegalMovesNN, self).__init__()
        
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=29, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=256 * 8 * 8, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=1858)
        
        # Define dropout for regularization
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        # Apply convolutional layers with ReLU activation
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)  # Reshape to (batch_size, 2048 * 8 * 8)
        
        # Apply fully connected layers with ReLU activation
        x = F.relu(self.fc1(x))
        
        # Apply dropout
        x = self.dropout(x)
        
        # Output layer
        x = self.fc2(x)
        
        return x
    
    def get_predicted_move(self, board: chess.Board) -> chess.Move:
        # Convert the board to a tensor
        if board.turn == chess.BLACK:
            raise ValueError("The board must be from white perspective!")
        board_tensor = chessboard_to_tensorboard_29x8x8(board)
        board_tensor = board_tensor.to(device)
        
        # Forward pass
        output = self.forward(board_tensor.unsqueeze(0))
        
        # Get the index of the move with the highest probability
        move_index = torch.argmax(output).item()
        
        # Convert the index to UCI move
        uci_move = get_uci_move_by_index(move_index)
        
        # Convert the UCI move to a chess move
        move = chess.Move.from_uci(uci_move)
        
        return move

    def play_move_on_chessboard(self, board: chess.Board) -> chess.Move:
        if board.turn == chess.BLACK:
            raise ValueError("The board must be from white perspective!")
        move = self.get_predicted_move(board)
        board.push(move)
        return move

# Split the dataset into training and validation sets
split = int(0.8 * len(board_tensors))

train_board_tensors = board_tensors[:split]
train_legal_moves_tensors = legal_moves_tensors[:split]

val_board_tensors = board_tensors[split:]
val_legal_moves_tensors = legal_moves_tensors[split:]

# Create the dataloaders
train_dataset = ChessDataset(train_board_tensors, train_legal_moves_tensors)
val_dataset = ChessDataset(val_board_tensors, val_legal_moves_tensors)

BATCH_SIZE = 32
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)


Using device: cuda


In [5]:
# Initialize the model, loss function, and optimizer
PATH_MODEL = "TorreEngineLegalMovesNN.pth"

# Create a new model
model = TorreEngineLegalMovesNN().to(device)

# Load the model dict from a file
#model.load_state_dict(torch.load(PATH_MODEL)) # UNCOMMENT TO LOAD THE MODEL FROM A FILE

criterion = nn.CrossEntropyLoss()  # Classification problem
#criterion = nn.BCEWithLogitsLoss()  # Binary classification problem
#criterion = nn.MSELoss()  # Regression problem

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2e-5)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=2e-5)

In [6]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (board_tensors, legal_moves_tensors) in enumerate(train_dataloader):
        board_tensors, legal_moves_tensors = board_tensors.to(device), legal_moves_tensors.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(board_tensors)
        
        # Compute the loss
        loss = criterion(outputs, legal_moves_tensors)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_dataloader)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, (board_tensors, legal_moves_tensors) in enumerate(val_dataloader):
            board_tensors, legal_moves_tensors = board_tensors.to(device), legal_moves_tensors.to(device)
            
            # Forward pass
            outputs = model(board_tensors)
            
            # Compute the loss
            loss = criterion(outputs, legal_moves_tensors)
            
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_dataloader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

Epoch [1/10], Train Loss: 113.1964, Val Loss: 91.1314
Epoch [2/10], Train Loss: 97.9814, Val Loss: 87.0102
Epoch [3/10], Train Loss: 94.2177, Val Loss: 84.7785
Epoch [4/10], Train Loss: 90.1379, Val Loss: 81.8059
Epoch [5/10], Train Loss: 86.3287, Val Loss: 79.9642
Epoch [6/10], Train Loss: 84.3753, Val Loss: 78.9096
Epoch [7/10], Train Loss: 83.1328, Val Loss: 78.2522
Epoch [8/10], Train Loss: 82.3365, Val Loss: 77.8063
Epoch [9/10], Train Loss: 81.8292, Val Loss: 77.5595
Epoch [10/10], Train Loss: 81.4859, Val Loss: 77.3822


In [7]:
# Save the model
torch.save(model.state_dict(), PATH_MODEL)

# Print model parameters count
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of parameters: 5057858


## Benchmark vs untrained model and random UCI moves

In [8]:
from torrechess.utils import get_uci_move_by_index

model_trained = TorreEngineLegalMovesNN().to(device)
model_trained.load_state_dict(torch.load(PATH_MODEL))
model_trained.eval()

model_untrained = TorreEngineLegalMovesNN().to(device)
model_untrained.eval()

# Evaluate how many legal moves are predicted correctly
import chess
import random

random_uci_move_score = 0
trained_score = 0
untrained_score = 0

total_attempts = 0

N = 15

for i in range(N):
    board = chess.Board()
    while not board.is_game_over():
        total_attempts += 0.5

        if board.turn == chess.WHITE: # Only predict moves for white as the model is trained for white perspective
            # Benchmark with random UCI move (out of all 1858)
            random_uci_move = chess.Move.from_uci(get_uci_move_by_index(random.randint(0, 1857)))
            if board.is_legal(random_uci_move):
                random_uci_move_score += 1

            # See if model_trained predicts a legal move in this position
            move_by_trained = model_trained.get_predicted_move(board)
            if board.is_legal(move_by_trained):
                trained_score += 1

            # See if model_untrained predicts a legal move in this position
            move_by_untrained = model_untrained.get_predicted_move(board)
            if board.is_legal(move_by_untrained):
                untrained_score += 1

        # Play a random move to advance the game
        random_legal_move = random.choice(list(board.legal_moves))
        board.push(random_legal_move)

print(f'Random UCI move predicted {random_uci_move_score} legal moves out of {total_attempts} ({random_uci_move_score/total_attempts:.2f})')
print(f'Trained model predicted {trained_score} legal moves out of {total_attempts} ({trained_score/total_attempts:.2f})')
print(f'Untrained model predicted {untrained_score} legal moves out of {total_attempts} ({untrained_score/total_attempts:.2f})')


Random UCI move predicted 35 legal moves out of 2419.5 (0.01)
Trained model predicted 2378 legal moves out of 2419.5 (0.98)
Untrained model predicted 22 legal moves out of 2419.5 (0.01)
